In [9]:
## IMPORTS
import random
from deap import base
from deap import creator
from deap import tools
import numpy as np
import timeit
start = timeit.default_timer()
from stage import fullPrint

In [10]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)


C:\Users\henry\miniconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\henry\miniconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [11]:
def generate_random_num_attr():
  original = np.array([0,0,0,0,0,0,0,0])
  noises = []
  sum_list = []

  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0,3))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.2))

  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list
noises = []
noises.append(np.random.normal(0, 0.025))
noises.append(np.random.normal(0, 0.075))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.0125))
noises.append(np.random.normal(0, 0.015))
noises.append(np.random.normal(0, 0.005))
noises.append(np.random.normal(0, 0.01))
def mutate(individual):
  size = len(individual)
  for i in range(0,size):
    if random.random() > 0.9:
      individual[i] = individual[i] + noises[i]
  return individual,

In [12]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", generate_random_num_attr)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [13]:
## change to our functinon
from stage import getStage
import numpy as np
import pandas as pd
from datetime import date
import yahoo_fin.stock_info as yf
import random
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)
def evalOneMax(individual):
    print(individual[0])
    transaction = yf.get_data('AAPL', start_date="1995-01-02",end_date="2022-01-17", index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in random.sample(allList,1000):
        df = getStage(symbol,individual[0])
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        for index, element in df.iterrows():
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0,
    return transactionCopy.iloc[-1]['total'],

In [14]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

In [15]:
def main():


    pop = toolbox.population(n=1024)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
        
    # Begin the evolution
    for g in range(10):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                toolbox.mate(child1[0], child2[0])
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant[0])
                del mutant.fitness.values        
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Replace population
        pop[:] = offspring
    
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))


In [16]:
main()

[0.6089378286276071, 1.8773967687982491, 0.11543263547491148, 0.9116681598025687, 1.059749856895917, 1.140809463176896, 0.8845341691276128, 1.099930612227392]


C:\Users\henry\AppData\Local\Temp/ipykernel_20220/2130801844.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,1000):


4.22041105598866
1956.885859645898
[0.7520689977076149, 1.78497768322871, 0.11572776468255178, 0.9099962189800969, 1.1671145535816259, 1.204271793710095, 0.8645362281032682, 1.0731850183462783]


C:\Users\henry\AppData\Local\Temp/ipykernel_20220/2130801844.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,1000):


2.258681785967399
2487.726850164791
-- Generation 0 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0
-- Generation 1 --
[0.5811926771449876, 1.8773967687982491, 0.11543263547491148, 0.9055398230475257, 1.059749856895917, 1.140809463176896, 0.8845341691276128, 1.099930612227392]


C:\Users\henry\AppData\Local\Temp/ipykernel_20220/2130801844.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,1000):


3.073706591070163
5586.016117793189
[0.6089378286276071, 1.8773967687982491, 0.11543263547491148, 0.9116681598025687, 1.059749856895917, 1.140809463176896, 0.8845341691276128, 1.099930612227392]


C:\Users\henry\AppData\Local\Temp/ipykernel_20220/2130801844.py:23: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for symbol in random.sample(allList,1000):
